## Import Functions and Data

In [ ]:
import sys


## Training Parameters
numAugs = 0
autoAugment = True
bayesianRun = False
testDataName='IEEEData'
maxEpochs= 40
train_split = 0
## Base Parameters
valPerc=.15
learning_rate =.0001
dim=224
lr_step_size = 10
batchsize=16
dropout=False

augRun=True
if numAugs==0:
    augRun=False
print(augRun)

## Import Torch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torchvision as torchvision
import matplotlib.pyplot as plt
import pandas
import time
import os
import copy
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from os import listdir
from os.path import isfile, join, isdir, islink,exists
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas


from torch.nn import functional as F

from PIL import Image
import sys

from sklearn.model_selection import train_test_split, StratifiedKFold

from torch.utils.data import DataLoader, WeightedRandomSampler

import cv2

from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

####

mix_name='None'  ## Use mixup, None, or cutmix  all will use Class Sampling as Published


## Import Data

In [ ]:
# Need Pointer to Downloaded KCapsule Dataset
# All images are downloaded unzipped in seperate folders by class
KCapsule='/project/GutIntelligenceLab/jaj4zcf/Videos/KCapsule'

split_0=pandas.read_csv(KCapsule+'/split_0.csv')
split_1=pandas.read_csv(KCapsule+'/split_1.csv')
KCclasses=[f for f in listdir(KCapsule) if '.' not in f]
kvasir_paper=True
if kvasir_paper==True:
    class_dict={'Erosion':'ulcer', 
     'Lymphangiectasia':'mass',
     'Blood - fresh':'bleeding',
     'Erythema':'inflammation',
     'Ulcer':'ulcer',
     'Angiectasia':'angioectasia',
     'Normal clean mucosa':'normal',
     'Ileocecal valve': 'ileocecal valve',
     'Pylorus': 'pylorus',
     'Reduced mucosal view': 'reduced mucosal view',
     'Foreign body':'foreign body'
               }

anatomy_list=[]
datas=[]

reclass=True 
for KCclass in KCclasses:
    if reclass:
        if KCclass not in class_dict:
            continue
    newData=pandas.DataFrame()
    #print(KCclass)
    classDir=KCapsule+'/'+KCclass
    fileNames=listdir(classDir)
    split0=[f in split_0['filename'].values for f in fileNames if '.jpg' in f]
    newData['split0']=split0
    files=[classDir + '/'+ f for f in fileNames if '.jpg' in f]
    newData['files']=files
    image_id=[KCclass + '/'+ f for f in fileNames if '.jpg' in f]
    newData['image_id']=image_id
    if reclass:
        newData['pathology']=class_dict[KCclass]
    else:
        newData['pathology']=KCclass
    datas.append(newData)
    #print(KCclass)
    #print(len(files))
KCapsData=pandas.concat(datas, ignore_index=True)
KCapsData.shape


# Extract Just Ulcers for This Experiment

labelsdf=KCapsData
labelsdf=labelsdf[(labelsdf.pathology=='normal')|(labelsdf.pathology=='ulcer' )]
print(np.sum(labelsdf.pathology=='ulcer'))
print('ulcers')
labelsdf

## Data Statistics For Review
pivot=pandas.pivot_table(KCapsData, values='files',index='split0',columns=['pathology'], aggfunc='count')


# keeping only small bowel abnormalities represented in both classes
pivot2=pivot.reset_index().melt(id_vars=['split0'])

pivot2=pivot2.sort_values('value',ascending=False)

In [ ]:
## Add An Index Per Class
def finddr(x):
    return np.where(x==np.sort(labelsdf.pathology.unique()))[0][0]
labelsdf['dr']=labelsdf.pathology.apply(finddr)
labelsdf

## CutMix and Mixup Codes

In [ ]:
def mixup_data(x, y, x2, y2, classes=2):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    
    lam = np.random.beta(0.2, 1)
        
    mixed_x = lam * x + (1 - lam) * x2
    y_out=np.zeros(classes)
    y_out[y] = y_out[y] + lam
    y_out[y2] = y_out[y2] + (1 - lam)
    
    return mixed_x, y_out

def rand_bbox(size, lam):
    W = size[0]
    H = size[1]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2


def cutmix_data(x, y, x2, y2, alpha=1.0,classes=2):
    # generate mixed sample
    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.shape, lam)
    x[bbx1:bbx2, bby1:bby2,:] = x2[bbx1:bbx2, bby1:bby2,:]
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.shape[0] * x.shape[1]))
    y_out=np.zeros(classes)
    y_out[y] = y_out[y] + lam
    y_out[y2] = y_out[y2] + (1 - lam)
    # compute output
    return x,y_out

In [ ]:
### Dataset
from PIL import Image
class Dataset(torch.utils.data.Dataset):
    def __init__(self, list_IDs, column='pathology', dims=(32,32,32), transform=False, mix='None'):
        'Initialization'
        self.dims = dims
        self.list_IDs = list_IDs
        self.n_classes = self.list_IDs[column].unique().shape[0]
        self.classes= np.array(sorted(self.list_IDs[column].unique()))
        self.i = 0
        self.column = column
        self.transform = transform
        self.mix = mix
            

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)
    
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        if torch.is_tensor(index):
            index = index.tolist()
        ID = self.list_IDs.loc[index]
        #print('here')
        #print(ID)    
        imCenterpath=ID['files']
        img = Image.open(imCenterpath)
        img = img.resize((self.dims[0],self.dims[1]))    
        y = np.where(self.classes==ID[self.column])[0][0]
        
        if self.mix == 'None' or np.random.random()>.5:  #apply cutmix or mixup 50% of the time
            y_out = np.zeros(self.n_classes)
            y_out[y]=1
        else:
            ## Get Another Image to Blend
            index1=np.random.randint(self.list_IDs.shape[0])
            ID = self.list_IDs.loc[index1]
            imCenterpath=ID['files']
            img2 = Image.open(imCenterpath)
            img2 = img2.resize((self.dims[0],self.dims[1]))
            y2 = np.where(self.classes==ID[self.column])[0][0]
            img = np.array(img)[:,:,:]
            img2 = np.array(img2)[:,:,:]
            if self.mix == 'cutmix':
                img, y_out = cutmix_data(img, y, img2, y2)  
            elif self.mix == 'mixup':
                img, y_out = mixup_data(img, y, img2, y2)
            img=Image.fromarray(img.astype('uint8'))
        if self.transform:
            #X = Image.fromarray(img)
            X = self.transform(img)            
            return X, y_out
        else:
            #X =  img /255
            #X=np.rollaxis(X, 2, 0)  
            return img, y_out

In [ ]:
##Split Training and Test by split0 and split1
if train_split==0:
    x_train=labelsdf[labelsdf.split0==False]
    x_test=labelsdf[labelsdf.split0==True]
else:
    x_train=labelsdf[labelsdf.split0==True]
    x_test=labelsdf[labelsdf.split0==False]
    
y_test=x_test
print(x_train.pathology.unique())
print(x_test.pathology.unique())
#x_train, x_test, y_train, y_test = train_test_split(labelsdf, labelsdf, random_state=12, test_size=testPerc, shuffle=True,stratify=labelsdf['pathology'])

In [ ]:
#y_test.to_csv('/project/DSone/jaj4zcf/Videos/models/'+testDataName+'.csv',index=False)
#make valid for training
x_train, x_val, y_train, y_val = train_test_split(x_train, x_train,random_state=13, test_size=valPerc, shuffle= True,stratify=x_train['pathology'])
abNormal=x_train[x_train['pathology']!='normal']

In [ ]:
y_train=y_train.reset_index(drop=True)
y_val=y_val.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)
x_train=x_train.reset_index(drop=True)
x_val=x_val.reset_index(drop=True)
x_test=x_test.reset_index(drop=True)

## Add Mask Index
Will be used later when doing gradient based image blend data augmnetation. 

In [ ]:
x_train['mask_index']=np.NaN
i = 0
for j in range(0,x_train.shape[0]):
    if x_train.pathology.loc[j]=='ulcer':
        x_train.mask_index.loc[j]=i
        i=i+1

In [ ]:
## Save Data Splits In Case Needed Later
x_train.to_csv('train_VCE0.csv',index=False)
x_test.to_csv('test_VCE0.csv',index=False)
x_val.to_csv('val_VCE0.csv',index=False)

In [ ]:
## Used to Generate Grad-CAMs (run through trainin data without augmentation)
x_train_no_reps=x_train.reset_index(drop=True)

## Create Data Augmentation Transforms

In [ ]:
if autoAugment==True:
     train_transform = transforms.Compose([
            #transforms.Pad(padding=42),
            transforms.RandomAffine(45, translate=(.2, .2), scale=(.8,1), shear=15),   
            transforms.ColorJitter(brightness=(.9,1.1), contrast=(0.8, 1.2), saturation=(0.9, 1.1)), 
            #transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.001, 5)),
            
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            #transforms.AutoAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    
else:
    train_transform = transforms.Compose([
            #transforms.Pad(padding=42),
            #transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

#Training and Validation With Oversampling
traingen=Dataset(x_train, column='pathology', dims=(dim,dim,3), transform=train_transform,mix = mix_name)
valgen=Dataset(y_val, column='pathology', dims=(dim,dim,3), transform=val_transform)

#Used to Create GRADCAMS
traingen2=Dataset(x_train, column='pathology', dims=(dim,dim,3), transform=val_transform)

#Test Dataset
testgen=Dataset(y_test, column='pathology', dims=(dim,dim,3), transform=val_transform)

#Test DataSet For Use With Test Time Augmentation
testgenAug=Dataset(y_test, column='pathology', dims=(dim,dim,3), transform=train_transform)

## Use Class Based Augmentation 

In [ ]:
params = {'batch_size': batchsize,
          'shuffle': True,
          'num_workers': 1}

traingenerator=torch.utils.data.DataLoader(traingen, **params)
valgenerator=torch.utils.data.DataLoader(valgen, **params)

testgenerator=torch.utils.data.DataLoader(testgen, **params)

## Code adopted from Balanced Mixup
## https://github.com/agaldran/balanced_mixup/
def get_sampling_probabilities(class_count, mode='instance', ep=None, n_eps=None):
    '''
    Note that for progressive sampling I use n_eps-1, which I find more intuitive.
    If you are training for 10 epochs, you pass n_eps=10 to this function. Then, inside
    the training loop you would have sth like 'for ep in range(n_eps)', so ep=0,...,9,
    and all fits together.
    '''
    if mode == 'instance':
        q = 0
    elif mode == 'class':
        q = 1
    elif mode == 'sqrt':
        q = 0.5 # 1/2
    elif mode == 'cbrt':
        q = 0.125 # 1/8
    elif mode == 'prog':
        assert ep != None and n_eps != None, 'progressive sampling requires to pass values for ep and n_eps'
        relative_freq_imbal = class_count ** 0 / (class_count ** 0).sum()
        relative_freq_bal = class_count ** 1 / (class_count ** 1).sum()
        sampling_probabilities_imbal = relative_freq_imbal ** (-1)
        sampling_probabilities_bal = relative_freq_bal ** (-1)
        return (1 - ep / (n_eps - 1)) * sampling_probabilities_imbal + (ep / (n_eps - 1)) * sampling_probabilities_bal
    else: sys.exit('not a valid mode')

    relative_freq = class_count ** q / (class_count ** q).sum()
    sampling_probabilities = relative_freq ** (-1)

    return sampling_probabilities

def returnsampler(classcol, mode, ep=None, n_eps=None):
    def retclass(x):
        return np.where(classes==x)[0][0]
    classes, class_count = np.unique(classcol, return_counts=True)
    print(class_count)
    sampling_probs = get_sampling_probabilities(class_count, mode=mode, ep=ep, n_eps=n_eps)
    print(sampling_probs)
    sample_weights = sampling_probs[classcol.apply(retclass)]
    
    mod_sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights))
    return mod_sampler
mod_sampler=returnsampler(traingen.list_IDs.pathology, mode='class')

params = {'batch_size': batchsize,
          'num_workers': 4,
         'sampler':mod_sampler}

modtraingenerator=torch.utils.data.DataLoader(traingen, **params)

In [ ]:
# Set Transform to False if You Want To See A Prview of Augmentations
#traingen.transform=False
img, y=traingen[12502]
print(y)
img

## Training Function

In [ ]:
num_batches = y_train.shape[0] / batchsize

MEAN = torch.tensor([0.485, 0.456, 0.406])
STD = torch.tensor([0.229, 0.224, 0.225])


def train_model(model, traingenerator, valgenerator, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    training_loss=[]
    val_loss=[]
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 100000


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                data=traingenerator
            else:
                data=valgenerator
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            
            for inputs, labels in data:
                inputs = inputs.to(device, dtype=torch.float)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    #print(outputs)
                    #print(labels)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                #print(torch.max(labels,1))
                running_corrects += torch.sum(preds == torch.max(labels, 1)[1])

            
            epoch_loss = running_loss / data.dataset.list_IDs.shape[0]
            epoch_acc = running_corrects.double() / data.dataset.list_IDs.shape[0]
            if phase == 'train':
                training_loss.append(epoch_loss)
                scheduler.step()
            else:
                val_loss.append(epoch_loss)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_acc = epoch_acc
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                #torch.save(model_ft.state_dict(),'/project/DSone/jaj4zcf/Videos/models/thirdsmidTrainBlendSplit'+str(train_split))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, training_loss, val_loss 

In [ ]:

def get_model():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #model_ft = models.resnet152(pretrained=True)
    model_ft = models.resnet18(pretrained=False)

    num_ftrs = model_ft.fc.in_features

    # Here the size of each output sample is set to 2.
    # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).


    if dropout==True:
        print('Dropout')
        model_ft.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs, 200),nn.ReLU(),nn.Dropout(0.5),nn.Linear(200, traingen.n_classes) )
    else:
        model_ft.fc = nn.Linear(num_ftrs, traingen.n_classes)


    model_ft.to(device)
    return model_ft, device
model_ft, device=get_model()

## Train Model
You must set a location accessible for your model. 

In [ ]:
criterion = nn.CrossEntropyLoss()

# In[ ]:
# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)

# Decay LR
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=lr_step_size, gamma=0.1)

## Need to set a Location for Your Model 
model_loc='/project/DSone/jaj4zcf/Videos/models/ResNet_'+mix_name+'_Base_Train_on_Split'+str(train_split)

In [ ]:
model_ft, training_loss, val_loss = train_model(model_ft, modtraingenerator, valgenerator, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=maxEpochs)
torch.save(model_ft.state_dict(),model_loc)

## Evaluate Model Prior to GradMix

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    
if torch.cuda.is_available():
    model_ft.cuda()
    
def test_model(model, data):
    was_training = model.training
    model.eval()
    images_so_far = 0
    origLabels=np.array([])
    predictions=np.array([])
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(data):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            _, labelsOut = torch.max(labels,1)
            predictions=np.append(predictions,np.array(preds.cpu()),axis=0)
            origLabels=np.append(origLabels,np.array(labelsOut.cpu()),axis=0)
            if i==0:
                outputs_full=np.array(outputs.cpu())
            else:
                outputs_full=np.append(outputs_full,np.array(outputs.cpu()),axis=0)
    model.train(mode=was_training)
    return origLabels, predictions, outputs_full


test_params = {'batch_size': 1,
          'shuffle': False,
          'num_workers': 3}

In [ ]:
#modelLoc='/project/DSone/jaj4zcf/Videos/models/ResNet_GradMix_Base_Train_on_Split'+str(train_split)
model_ft, device_2=get_model()

testgenerator=torch.utils.data.DataLoader(testgen, **test_params)
model_ft.load_state_dict(torch.load(model_loc))

In [ ]:
print('Loaded')
labels, preds,outputs_full=test_model(model_ft, testgenerator)
y_test['labels']=labels

y_test['preds_']=preds

In [ ]:
y_test.pathology.unique()
mix_name

In [ ]:
massout=y_test[(y_test.pathology=='normal')]
TN = np.sum(massout['preds_']==massout['labels'])
FP = np.sum(massout['preds_']!=massout['labels'])


massout=y_test[(y_test.pathology=='ulcer')]

TP = np.sum(massout['preds_']!=0)
FN = np.sum(massout['preds_']==0)

## Overall Specificity
print('accuracy')
print(np.sum(y_test['preds_']==y_test['labels'])/y_test.shape[0])
# True Postivies
print('sensitivity')
print(TP/(TP+FN))
print('specificity')
print(TN/(TN+FP))


labels_base=labels
preds_base=preds
outputs_ful_base=outputs_full
probs=[]
for l in range(0,len(outputs_full)):
    probs.append(np.exp(outputs_full[l])/np.sum(np.exp(outputs_full[l])))
probs_base=probs
print(accuracy_score(y_test['labels'],y_test['preds_']))
print(precision_score(y_test['labels'],y_test['preds_']))
print(recall_score(y_test['labels'],y_test['preds_']))
print(f1_score(y_test['labels'],y_test['preds_']))
roc_auc_score(labels,np.array(probs_base)[:,1])

## Class to Augment with Blended Image

In [ ]:
from PIL import Image

class BlendDataset(torch.utils.data.Dataset):
    def __init__(self, list_IDs, test_column='pathology', masks=[], mask_classes=[], dims=(32,32,32), transform=False):
        'Initialization'
        self.dims = dims
        self.list_IDs = list_IDs
        self.n_classes = self.list_IDs[test_column].unique().shape[0]
        self.classes= np.array(sorted(self.list_IDs[test_column].unique()))
        self.i = 0    
        self.transform = transform
        self.masks=masks
        self.mask_classes=mask_classes
        self.test_column = test_column
        print(self.mask_classes)
        
    
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)
    
    def get_blend(self, ulcer_forground,masks,mask_num, lam = 1):
        #print('blended')
        
        mask=masks[mask_num]
        index1=np.random.randint(self.list_IDs.shape[0])
        ID = self.list_IDs.loc[index1]
        imCenterpath=ID['files']
        #print(imCenterpath)
        norm_background = Image.open(imCenterpath)
        #print(ID)
        y2 = np.where(self.classes==ID[self.test_column])[0][0]    
        out_img = (mask[...,None] * np.array(ulcer_forground)[:,:,:]) + ((1-mask[...,None]) * np.array(norm_background)[:,:,:])
        out_img=Image.fromarray(out_img.astype('uint8'))
        y_out=np.zeros(self.n_classes)
        y_out[1] = lam  #make 75% chance if it is abnormal
        y_out[y2] = y_out[y2] + (1-lam)  # + 25% to either class depending on background
        return out_img, y_out   
    
    
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        if torch.is_tensor(index):
            index = index.tolist()
        ID = self.list_IDs.loc[index]
        #print('here')
        #print(ID)    
        imCenterpath=ID['files']
        img = Image.open(imCenterpath)
        y = np.where(self.classes==ID[self.test_column])[0][0]  
        y_out = np.zeros(self.n_classes)
        y_out[y]=1
        if not np.isnan(x_train.loc[index].mask_index):
            if np.random.random()>.5: # augment 50% of images
                mask_class=x_train.loc[index].pathology
                class_num=np.where(np.array(self.mask_classes)==mask_class)[0][0]
                img, y_out = self.get_blend(img,self.masks[class_num],int(x_train.loc[index].mask_index))
        
        img = img.resize((self.dims[0],self.dims[1]))    

        
        
        if self.transform:
            #X = Image.fromarray(img)
            
            X = self.transform(img)            
            return X, y_out
        else:
            #X =  img /255
            #X=np.rollaxis(X, 2, 0)  
            return img, y_out

## Create GradCAMs

In [ ]:
#ulcers_val=np.where(x_train_no_reps.pathology=='ulcer')[0]
import time
cam = GradCAMPlusPlus(model=model_ft, target_layer=model_ft.layer4[-1], use_cuda=True)

ulcer_cams=[]
num_cams = len(np.where(x_train_no_reps.pathology=='ulcer')[0])
i=0
for ulcer in np.where(x_train_no_reps.pathology=='ulcer')[0]:
    i=i+1
    img, _ =traingen2[ulcer]
    grayscale_cam=cam(img[None,:,:,:],target_category= [1] ,aug_smooth=True)
    ulcer_cams.append(cv2.resize(grayscale_cam[0, :],(336,336)))
    if i % 50 == 0:
        print(i/num_cams)
    

## Create Blending Data Generator

In [ ]:
BlendTrain=BlendDataset(x_train, test_column='pathology',masks=[ulcer_cams], mask_classes=['ulcer'], dims=(dim,dim,3), transform=train_transform)  
traingenerator=torch.utils.data.DataLoader(BlendTrain, **params)

In [ ]:
model_ft, device_2=get_model()


criterion = nn.CrossEntropyLoss()

# In[ ]:
# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)

# Decay LR
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=lr_step_size, gamma=0.1)

# In[ ]:
model_ft, training_loss, val_loss = train_model(model_ft, traingenerator, valgenerator, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=maxEpochs)

In [ ]:
modelLoc='/project/DSone/jaj4zcf/Videos/models/ResNet_GradMix_Blended_mixup_Train_on_SplitLAM'+str(train_split)
torch.save(model_ft.state_dict(),modelLoc)


## Load Trained Model and Test

In [ ]:
model_ft, device_2=get_model()
model_ft.load_state_dict(torch.load(modelLoc))
print('Loaded')
testgenerator=torch.utils.data.DataLoader(testgen, **test_params)
labels, preds,outputs_full=test_model(model_ft, testgenerator)

In [ ]:
y_test['labels']=labels
y_test['preds_']=preds
massout=y_test[(y_test.pathology=='normal')]
TN = np.sum(massout['preds_']==massout['labels'])
FP = np.sum(massout['preds_']!=massout['labels'])

massout=y_test[(y_test.pathology=='ulcer')]

TP = np.sum(massout['preds_']!=0)
FN = np.sum(massout['preds_']==0)

## Overall Specificity
print('accuracy')
print(np.sum(y_test['preds_']==y_test['labels'])/y_test.shape[0])
# True Postivies
print('sensitivity')
print(TP/(TP+FN))
print('specificity')
print(TN/(TN+FP))
# for SB abnormal Class
probs=[]
for l in range(0,len(outputs_full)):
    probs.append(np.exp(outputs_full[l])/np.sum(np.exp(outputs_full[l])))

print(accuracy_score(y_test['labels'],y_test['preds_']))
print(precision_score(y_test['labels'],y_test['preds_']))
print(recall_score(y_test['labels'],y_test['preds_']))
print(f1_score(y_test['labels'],y_test['preds_']))
roc_auc_score(labels,np.array(probs)[:,1])

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

class_num=0

ulc_fpr, ulc_tpr, _=roc_curve(labels, np.array(probs)[:,class_num])
    
ulc_fprB, ulc_tprB, _=roc_curve(labels, np.array(probs_base)[:,class_num]) 

from matplotlib import pyplot
# plot the roc curve for the model
fig = pyplot.figure(figsize=(4, 4))
pyplot.plot(ulc_fprB, ulc_tprB, linestyle='--', label='Base')
pyplot.plot(ulc_fpr, ulc_tpr, linestyle='-.', label='GradMix')

# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend

pyplot.legend(loc='lower right')
# show the plot
#pyplot.tight_layout()
pyplot.axis('equal')
pyplot.show()